In [1]:
import pandas as pd

from master_thesis.core.utils.reproducibility import seed_everything
from master_thesis.core.models.llama import load_model_and_tokenizer
from master_thesis.core.utils.prompts import load_prompt
from master_thesis.core.methods.probing_with_interventions import (
    ProbingWithInterventions,
)
from master_thesis.core.probes import load_cav
from master_thesis.core.utils.reproducibility import save_results


DEVICE = "cuda"
PROMPT_TYPE = "few_shot"
POSITIVE_TOKEN = "true"
NEGATIVE_TOKEN = "false"
MODEL = "LLAMA_2_7B_CHAT"

LABEL_ASPECT_MODEL = "LRProbe"
LABEL_ASPECT_LAYER = 24

CONFOUNDING_ASPECT_MODEL = "LRProbe"
CONFOUNDING_ASPECT_LAYER = 22

DATA_DIR = "../../../../data"
DATASETS_DIR = f"{DATA_DIR}/datasets/base_experiments/car_vs_bike/test"
CACHE_DIR = f".cache/results"
CLASS_NAMES = ["bike", "car"]
LABEL_ASCPECT_NAMES = ["bike", "car"]
CONFOUNDING_ASPECT_NAMES = ["negative", "positive"]

In [2]:
seed_everything()

In [3]:
model, tokenizer = load_model_and_tokenizer(MODEL)

/home/mateusz_politycki/master-thesis-probing/.venv/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:732: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/mateusz_politycki/master-thesis-probing/.venv/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
TOKENS = [tokenizer.encode(NEGATIVE_TOKEN)[-1], tokenizer.encode(POSITIVE_TOKEN)[-1]]

### Prepare datasets, prompts and cavs

In [5]:
test_50_50 = pd.read_csv(f"{DATASETS_DIR}/test_50_50.csv")
test_95_5 = pd.read_csv(f"{DATASETS_DIR}/test_95_5.csv")

In [6]:
label_prompt = load_prompt(
    DATA_DIR,
    dataset_path="base_experiments/car_vs_bike",
    prompt_type=PROMPT_TYPE,
    prompt_aspect="label_aspect",
)

confounding_prompt = load_prompt(
    DATA_DIR,
    dataset_path="base_experiments/car_vs_bike",
    prompt_type=PROMPT_TYPE,
    prompt_aspect="confounding_aspect",
)

In [7]:
label_cav = load_cav(f".cache/cavs/{LABEL_ASPECT_MODEL}", f"label_{LABEL_ASPECT_LAYER}")
confounding_cav = load_cav(
    f".cache/cavs/{CONFOUNDING_ASPECT_MODEL}", f"confounding_{CONFOUNDING_ASPECT_LAYER}"
)

### Estimate Probing With Interventions

### Label

#### Test 50 50

In [8]:
seed_everything()

probing_with_interventions = ProbingWithInterventions(
    cav=label_cav,
    model=model,
    tokenizer=tokenizer,
    tokens=TOKENS,
    aspect_prompt=label_prompt,
    prompt=label_prompt,
    layer=LABEL_ASPECT_LAYER,
    device=DEVICE,
    verbose=True,
)

In [9]:
test_50_50_results_label = probing_with_interventions.evaluate(
    test_50_50, aspect_label_name="label", original_label_name="label"
)

  0%|          | 0/200 [00:00<?, ?it/s]

alpha: 0, tick: 1 diff: -0.8009106516838074, tot: 0.9998140931129456, proba_true: 0.09945172071456909, proba_false: 0.9003623723983765
alpha: -0.5, tick: 0.5 diff: -0.5212345570325851, tot: 0.9998719245195389, proba_true: 0.23931868374347687, proba_false: 0.760553240776062
alpha: -0.75, tick: 0.25 diff: -0.23014870285987854, tot: 0.9998827874660492, proba_true: 0.3848670423030853, proba_false: 0.6150157451629639


  0%|          | 1/200 [00:03<10:55,  3.29s/it]

alpha: -0.875, tick: 0.125 diff: -0.1549730896949768, tot: 0.9998859763145447, proba_true: 0.42245644330978394, proba_false: 0.5774295330047607
Skipping
alpha: 0, tick: 1 diff: 0.9997966184891993, tot: 0.9999439820967382, proba_true: 0.9998703002929688, proba_false: 7.368180376943201e-05
alpha: 1, tick: 1 diff: 0.998204312985763, tot: 0.9998894112650305, proba_true: 0.9990468621253967, proba_false: 0.0008425491396337748
alpha: 2, tick: 1 diff: 0.9743565926328301, tot: 0.9995962968096137, proba_true: 0.9869764447212219, proba_false: 0.01261985208839178
alpha: 3, tick: 1 diff: 0.6238102316856384, tot: 0.9969245791435242, proba_true: 0.8103674054145813, proba_false: 0.18655717372894287
alpha: 4, tick: 1 diff: -0.7058124542236328, tot: 0.9761242866516113, proba_true: 0.13515591621398926, proba_false: 0.8409683704376221
alpha: 3.5, tick: 0.5 diff: -0.1458921730518341, tot: 0.9900605380535126, proba_true: 0.42208418250083923, proba_false: 0.5679763555526733
alpha: 3.25, tick: 0.25 diff: 0.29

  1%|          | 2/200 [00:08<13:57,  4.23s/it]

...

=== Label: 1, proba true:0.39115554094314575, proba false:0.46450868248939514, change: 1
alpha: 0, tick: 1 diff: -0.9976348996860906, tot: 0.9995735763804987, proba_true: 0.0009693383472040296, proba_false: 0.9986042380332947
alpha: 1, tick: 1 diff: -0.9741387385874987, tot: 0.9997754078358412, proba_true: 0.012818334624171257, proba_false: 0.9869570732116699
alpha: 2, tick: 1 diff: -0.7711350172758102, tot: 0.9998341351747513, proba_true: 0.11434955894947052, proba_false: 0.8854845762252808
alpha: 3, tick: 1 diff: 0.14733874797821045, tot: 0.9998774528503418, proba_true: 0.5736081004142761, proba_false: 0.4262693524360657
alpha: 2.5, tick: 0.5 diff: -0.35830163955688477, tot: 0.9998444318771362, proba_true: 0.32077139616012573, proba_false: 0.6790730357170105
alpha: 3.0, tick: 0.5 diff: 0.14733874797821045, tot: 0.9998774528503418, proba_true: 0.5736081004142761, proba_false: 0.4262693524360657
alpha: 2.75, tick: 0.25 diff: -0.1853051483631134, tot: 0.9998486936092377, proba_true: 0.4

100%|█████████▉| 199/200 [14:28<00:04,  4.73s/it]

=== Label: 0, proba true:0.5736081004142761, proba false:0.4262693524360657, change: 1
alpha: 0, tick: 1 diff: -0.9993668433598941, tot: 0.9998135927744443, proba_true: 0.00022337470727507025, proba_false: 0.9995902180671692
alpha: 1, tick: 1 diff: -0.9922403078526258, tot: 0.9998884219676256, proba_true: 0.0038240570574998856, proba_false: 0.9960643649101257
alpha: 2, tick: 1 diff: -0.88988471403718, tot: 0.9998823367059231, proba_true: 0.05499881133437157, proba_false: 0.9448835253715515
alpha: 3, tick: 1 diff: -0.21530210971832275, tot: 0.9998874664306641, proba_true: 0.39229267835617065, proba_false: 0.6075947880744934
alpha: 4, tick: 1 diff: 0.7895088121294975, tot: 0.9999111965298653, proba_true: 0.8947100043296814, proba_false: 0.10520119220018387
alpha: 3.5, tick: 0.5 diff: 0.4179959297180176, tot: 0.9998986721038818, proba_true: 0.7089473009109497, proba_false: 0.29095137119293213
alpha: 3.25, tick: 0.25 diff: 0.07795727252960205, tot: 0.9998825788497925, proba_true: 0.5389199

100%|██████████| 200/200 [14:33<00:00,  4.37s/it]

=== Label: 0, proba true:0.5389199256896973, proba false:0.4609626531600952, change: 1


In [10]:
save_results(
    test_50_50_results_label,
    class_names=CLASS_NAMES,
    aspect_names=LABEL_ASCPECT_NAMES,
    save_path=f"{CACHE_DIR}/probing_with_interventions/test_50_50/label_aspect_{LABEL_ASPECT_MODEL}_{LABEL_ASPECT_LAYER}.json",
)

In [11]:
test_50_50_results_label

{0: {0: 1.0, 1: -1.0}, 1: {0: -0.88, 1: 0.88}}

#### Test 95 5

In [12]:
seed_everything()

probing_with_interventions = ProbingWithInterventions(
    cav=label_cav,
    model=model,
    tokenizer=tokenizer,
    tokens=TOKENS,
    aspect_prompt=label_prompt,
    prompt=label_prompt,
    layer=LABEL_ASPECT_LAYER,
    device=DEVICE,
    verbose=True,
)

In [13]:
test_95_5_results_label = probing_with_interventions.evaluate(
    test_95_5, aspect_label_name="label", original_label_name="label"
)

  0%|          | 0/200 [00:00<?, ?it/s]

alpha: 0, tick: 1 diff: 0.9439286477863789, tot: 0.9999403469264507, proba_true: 0.9719344973564148, proba_false: 0.028005849570035934
alpha: 1, tick: 1 diff: 0.43711593747138977, tot: 0.9998333156108856, proba_true: 0.7184746265411377, proba_false: 0.2813586890697479
alpha: 2, tick: 1 diff: -0.7225737422704697, tot: 0.999304786324501, proba_true: 0.13836552202701569, proba_false: 0.8609392642974854
alpha: 1.5, tick: 0.5 diff: -0.1852656900882721, tot: 0.9996357858181, proba_true: 0.40718504786491394, proba_false: 0.592450737953186
alpha: 1.25, tick: 0.25 diff: 0.08570429682731628, tot: 0.9997403919696808, proba_true: 0.5427223443984985, proba_false: 0.45701804757118225
alpha: 1.5, tick: 0.25 diff: -0.1852656900882721, tot: 0.9996357858181, proba_true: 0.40718504786491394, proba_false: 0.592450737953186
alpha: 1.375, tick: 0.125 diff: -0.007809937000274658, tot: 0.9996967911720276, proba_true: 0.49594342708587646, proba_false: 0.5037533640861511


  0%|          | 1/200 [00:03<12:10,  3.67s/it]

=== Label: 1, proba true:0.49594342708587646, proba false:0.5037533640861511, change: 1
alpha: 0, tick: 1 diff: 0.9997966184891993, tot: 0.9999439820967382, proba_true: 0.9998703002929688, proba_false: 7.368180376943201e-05
alpha: 1, tick: 1 diff: 0.998204312985763, tot: 0.9998894112650305, proba_true: 0.9990468621253967, proba_false: 0.0008425491396337748
alpha: 2, tick: 1 diff: 0.9743565926328301, tot: 0.9995962968096137, proba_true: 0.9869764447212219, proba_false: 0.01261985208839178
alpha: 3, tick: 1 diff: 0.6238102316856384, tot: 0.9969245791435242, proba_true: 0.8103674054145813, proba_false: 0.18655717372894287
alpha: 4, tick: 1 diff: -0.7058124542236328, tot: 0.9761242866516113, proba_true: 0.13515591621398926, proba_false: 0.8409683704376221
alpha: 3.5, tick: 0.5 diff: -0.1458921730518341, tot: 0.9900605380535126, proba_true: 0.42208418250083923, proba_false: 0.5679763555526733
alpha: 3.25, tick: 0.25 diff: 0.2940232455730438, tot: 0.9946778118610382, proba_true: 0.6443505287

  1%|          | 2/200 [00:08<14:30,  4.40s/it]

...

=== Label: 1, proba true:0.4461638629436493, proba false:0.5466513633728027, change: 1
alpha: 0, tick: 1 diff: -0.9986353706335649, tot: 0.9997406412148848, proba_true: 0.0005526352906599641, proba_false: 0.9991880059242249
alpha: 1, tick: 1 diff: -0.9834784874692559, tot: 0.9998518815264106, proba_true: 0.008186697028577328, proba_false: 0.9916651844978333
alpha: 2, tick: 1 diff: -0.8296957984566689, tot: 0.9998721107840538, proba_true: 0.08508815616369247, proba_false: 0.9147839546203613
alpha: 3, tick: 1 diff: 0.07018876075744629, tot: 0.9998843669891357, proba_true: 0.535036563873291, proba_false: 0.4648478031158447
alpha: 2.5, tick: 0.5 diff: -0.48626595735549927, tot: 0.9998559355735779, proba_true: 0.2567949891090393, proba_false: 0.7430609464645386
alpha: 3.0, tick: 0.5 diff: 0.07018876075744629, tot: 0.9998843669891357, proba_true: 0.535036563873291, proba_false: 0.4648478031158447
alpha: 2.75, tick: 0.25 diff: -0.2595154941082001, tot: 0.9998699724674225, proba_true: 0.370177

100%|█████████▉| 199/200 [14:45<00:04,  4.81s/it]

=== Label: 0, proba true:0.535036563873291, proba false:0.4648478031158447, change: 1
alpha: 0, tick: 1 diff: -0.9977038325741887, tot: 0.9996732054278255, proba_true: 0.0009846864268183708, proba_false: 0.9986885190010071
alpha: 1, tick: 1 diff: -0.9741655653342605, tot: 0.9998029405251145, proba_true: 0.012818687595427036, proba_false: 0.9869842529296875
alpha: 2, tick: 1 diff: -0.7679485827684402, tot: 0.9998307675123215, proba_true: 0.11594109237194061, proba_false: 0.8838896751403809
alpha: 3, tick: 1 diff: 0.1625848114490509, tot: 0.9998687207698822, proba_true: 0.5812267661094666, proba_false: 0.41864195466041565
alpha: 2.5, tick: 0.5 diff: -0.43712037801742554, tot: 0.9998435378074646, proba_true: 0.28136157989501953, proba_false: 0.7184819579124451
alpha: 3.0, tick: 0.5 diff: 0.1625848114490509, tot: 0.9998687207698822, proba_true: 0.5812267661094666, proba_false: 0.41864195466041565
alpha: 2.75, tick: 0.25 diff: -0.17017802596092224, tot: 0.9998572170734406, proba_true: 0.414

100%|██████████| 200/200 [14:50<00:00,  4.45s/it]

=== Label: 0, proba true:0.5233560800552368, proba false:0.4765211343765259, change: 1


In [14]:
save_results(
    test_95_5_results_label,
    class_names=CLASS_NAMES,
    aspect_names=LABEL_ASCPECT_NAMES,
    save_path=f"{CACHE_DIR}/probing_with_interventions/test_95_5/label_aspect_{LABEL_ASPECT_MODEL}_{LABEL_ASPECT_LAYER}.json",
)

In [15]:
test_95_5_results_label

{0: {0: 1.0, 1: -1.0}, 1: {0: -0.97, 1: 0.97}}

### Confounding

#### Test 50 50

In [16]:
seed_everything()

probing_with_interventions = ProbingWithInterventions(
    cav=confounding_cav,
    model=model,
    tokenizer=tokenizer,
    tokens=TOKENS,
    aspect_prompt=confounding_prompt,
    prompt=label_prompt,
    layer=CONFOUNDING_ASPECT_LAYER,
    device=DEVICE,
    verbose=True,
)

In [17]:
test_50_50_results_confounding = probing_with_interventions.evaluate(
    test_50_50, aspect_label_name="confounding", original_label_name="label"
)

  0%|          | 0/200 [00:00<?, ?it/s]

alpha: 0, tick: 1 diff: -0.9991656740630788, tot: 0.9991912642990428, proba_true: 1.279511798202293e-05, proba_false: 0.9991784691810608
alpha: 1, tick: 1 diff: -0.7834159135818481, tot: 0.9997369050979614, proba_true: 0.10816049575805664, proba_false: 0.8915764093399048
alpha: 2, tick: 1 diff: 0.9987723613157868, tot: 0.999949119053781, proba_true: 0.9993607401847839, proba_false: 0.000588378868997097
alpha: 1.5, tick: 0.5 diff: 0.8504047989845276, tot: 0.9999352097511292, proba_true: 0.9251700043678284, proba_false: 0.07476520538330078
alpha: 1.25, tick: 0.25 diff: 0.02342963218688965, tot: 0.9998478889465332, proba_true: 0.5116387605667114, proba_false: 0.4882091283798218
alpha: 1.125, tick: 0.125 diff: -0.5268575251102448, tot: 0.9997912347316742, proba_true: 0.23646685481071472, proba_false: 0.7633243799209595
alpha: 1.25, tick: 0.125 diff: 0.02342963218688965, tot: 0.9998478889465332, proba_true: 0.5116387605667114, proba_false: 0.4882091283798218


  0%|          | 1/200 [00:03<12:20,  3.72s/it]

=== Label: 1, proba true:0.999679446220398, proba false:0.00029136156081222, change: 0
alpha: 0, tick: 1 diff: 0.9998388176973094, tot: 0.9999816531126271, proba_true: 0.9999102354049683, proba_false: 7.141770765883848e-05
alpha: 1, tick: 1 diff: -0.34461545944213867, tot: 0.9998608827590942, proba_true: 0.3276227116584778, proba_false: 0.6722381711006165
alpha: 0.5, tick: 0.5 diff: 0.9718383457511663, tot: 0.9999646041542292, proba_true: 0.9859014749526978, proba_false: 0.01406312920153141
alpha: 1.0, tick: 0.5 diff: -0.34461545944213867, tot: 0.9998608827590942, proba_true: 0.3276227116584778, proba_false: 0.6722381711006165
alpha: 0.75, tick: 0.25 diff: 0.6622413247823715, tot: 0.9999324232339859, proba_true: 0.8310868740081787, proba_false: 0.1688455492258072
alpha: 1.0, tick: 0.25 diff: -0.34461545944213867, tot: 0.9998608827590942, proba_true: 0.3276227116584778, proba_false: 0.6722381711006165
alpha: 0.875, tick: 0.125 diff: 0.18531346321105957, tot: 0.9998935461044312, proba_tr

  1%|          | 2/200 [00:07<12:49,  3.89s/it]

...

=== Label: 1, proba true:0.999843955039978, proba false:1.1902132057706183e-10, change: 0
alpha: 0, tick: 1 diff: -0.9996560408917503, tot: 0.9996618435534401, proba_true: 2.9013308449066244e-06, proba_false: 0.9996589422225952
alpha: 1, tick: 1 diff: -0.9375931397080421, tot: 0.9998728856444359, proba_true: 0.03113987296819687, proba_false: 0.968733012676239
alpha: 2, tick: 1 diff: 0.9970929643604904, tot: 0.9999579170253128, proba_true: 0.9985254406929016, proba_false: 0.00143247633241117
alpha: 1.5, tick: 0.5 diff: 0.6443419307470322, tot: 0.9999436289072037, proba_true: 0.8221427798271179, proba_false: 0.17780084908008575
alpha: 1.25, tick: 0.25 diff: -0.5491047501564026, tot: 0.9998870491981506, proba_true: 0.22539114952087402, proba_false: 0.7744958996772766
alpha: 1.5, tick: 0.25 diff: 0.6443419307470322, tot: 0.9999436289072037, proba_true: 0.8221427798271179, proba_false: 0.17780084908008575
alpha: 1.375, tick: 0.125 diff: 0.1320255994796753, tot: 0.9999130964279175, proba_tru

100%|█████████▉| 199/200 [12:15<00:03,  3.73s/it]

=== Label: 0, proba true:0.9838051199913025, proba false:0.016152143478393555, change: 1
alpha: 0, tick: 1 diff: -0.9994916343202931, tot: 0.9995260811338085, proba_true: 1.7223406757693738e-05, proba_false: 0.9995088577270508
alpha: 1, tick: 1 diff: -0.7773924171924591, tot: 0.9998390376567841, proba_true: 0.11122331023216248, proba_false: 0.8886157274246216
alpha: 2, tick: 1 diff: 0.9988851239904761, tot: 0.9999566385522485, proba_true: 0.9994208812713623, proba_false: 0.0005357572808861732
alpha: 1.5, tick: 0.5 diff: 0.897794209420681, tot: 0.9999550953507423, proba_true: 0.9488746523857117, proba_false: 0.05108044296503067
alpha: 1.25, tick: 0.25 diff: 0.20784196257591248, tot: 0.9998957216739655, proba_true: 0.603868842124939, proba_false: 0.3960268795490265
alpha: 1.125, tick: 0.125 diff: -0.4742507040500641, tot: 0.9998616874217987, proba_true: 0.2628054916858673, proba_false: 0.7370561957359314
alpha: 1.25, tick: 0.125 diff: 0.20784196257591248, tot: 0.9998957216739655, proba_t

100%|██████████| 200/200 [12:19<00:00,  3.70s/it]

=== Label: 0, proba true:0.9304307103157043, proba false:0.06953968852758408, change: 1


In [18]:
save_results(
    test_50_50_results_confounding,
    class_names=CLASS_NAMES,
    aspect_names=CONFOUNDING_ASPECT_NAMES,
    save_path=f"{CACHE_DIR}/probing_with_interventions/test_50_50/confounding_aspect_{CONFOUNDING_ASPECT_MODEL}_{CONFOUNDING_ASPECT_LAYER}.json",
)

In [19]:
test_50_50_results_confounding

{0: {0: 0.47, 1: -0.47}, 1: {0: -0.27, 1: 0.27}}

#### Test 95 5

In [20]:
seed_everything()

probing_with_interventions = ProbingWithInterventions(
    cav=confounding_cav,
    model=model,
    tokenizer=tokenizer,
    tokens=TOKENS,
    aspect_prompt=confounding_prompt,
    prompt=label_prompt,
    layer=CONFOUNDING_ASPECT_LAYER,
    device=DEVICE,
    verbose=True,
)

In [21]:
test_95_5_results_confounding = probing_with_interventions.evaluate(
    test_95_5, aspect_label_name="confounding", original_label_name="label"
)

  0%|          | 0/200 [00:00<?, ?it/s]

alpha: 0, tick: 1 diff: 0.9999362388643931, tot: 0.9999729236569692, proba_true: 0.9999545812606812, proba_false: 1.8342396288062446e-05
alpha: 1, tick: 1 diff: -0.20032918453216553, tot: 0.999763011932373, proba_true: 0.39971691370010376, proba_false: 0.6000460982322693
alpha: 0.5, tick: 0.5 diff: 0.9861493622884154, tot: 0.9999564373865724, proba_true: 0.9930528998374939, proba_false: 0.0069035375490784645
alpha: 1.0, tick: 0.5 diff: -0.20032918453216553, tot: 0.999763011932373, proba_true: 0.39971691370010376, proba_false: 0.6000460982322693
alpha: 0.75, tick: 0.25 diff: 0.8248090296983719, tot: 0.9999232739210129, proba_true: 0.9123661518096924, proba_false: 0.0875571221113205
alpha: 1.0, tick: 0.25 diff: -0.20032918453216553, tot: 0.999763011932373, proba_true: 0.39971691370010376, proba_false: 0.6000460982322693
alpha: 0.875, tick: 0.125 diff: 0.4244019687175751, tot: 0.9998530447483063, proba_true: 0.7121275067329407, proba_false: 0.2877255380153656
alpha: 1.0, tick: 0.125 diff:

  0%|          | 1/200 [00:04<14:29,  4.37s/it]

=== Label: 1, proba true:0.00844116136431694, proba false:0.9910300970077515, change: 1
alpha: 0, tick: 1 diff: 0.9998388176973094, tot: 0.9999816531126271, proba_true: 0.9999102354049683, proba_false: 7.141770765883848e-05
alpha: 1, tick: 1 diff: -0.34461545944213867, tot: 0.9998608827590942, proba_true: 0.3276227116584778, proba_false: 0.6722381711006165
alpha: 0.5, tick: 0.5 diff: 0.9718383457511663, tot: 0.9999646041542292, proba_true: 0.9859014749526978, proba_false: 0.01406312920153141
alpha: 1.0, tick: 0.5 diff: -0.34461545944213867, tot: 0.9998608827590942, proba_true: 0.3276227116584778, proba_false: 0.6722381711006165
alpha: 0.75, tick: 0.25 diff: 0.6622413247823715, tot: 0.9999324232339859, proba_true: 0.8310868740081787, proba_false: 0.1688455492258072
alpha: 1.0, tick: 0.25 diff: -0.34461545944213867, tot: 0.9998608827590942, proba_true: 0.3276227116584778, proba_false: 0.6722381711006165
alpha: 0.875, tick: 0.125 diff: 0.18531346321105957, tot: 0.9998935461044312, proba_t

  1%|          | 2/200 [00:08<13:44,  4.16s/it]

...

=== Label: 1, proba true:0.7399397492408752, proba false:0.2597433030605316, change: 0
alpha: 0, tick: 1 diff: -0.9989750515669584, tot: 0.9995854962617159, proba_true: 0.0003052223473787308, proba_false: 0.9992802739143372
alpha: 1, tick: 1 diff: 0.16258996725082397, tot: 0.9999004006385803, proba_true: 0.5812451839447021, proba_false: 0.4186552166938782
alpha: 0.5, tick: 0.5 diff: -0.9648928083479404, tot: 0.9997697435319424, proba_true: 0.01743846759200096, proba_false: 0.9823312759399414
alpha: 1.0, tick: 0.5 diff: 0.16258996725082397, tot: 0.9999004006385803, proba_true: 0.5812451839447021, proba_false: 0.4186552166938782
alpha: 0.75, tick: 0.25 diff: -0.7548393905162811, tot: 0.9998487532138824, proba_true: 0.12250468134880066, proba_false: 0.8773440718650818
alpha: 1.0, tick: 0.25 diff: 0.16258996725082397, tot: 0.9999004006385803, proba_true: 0.5812451839447021, proba_false: 0.4186552166938782
alpha: 0.875, tick: 0.125 diff: -0.3718498647212982, tot: 0.9998681843280792, proba_t

100%|█████████▉| 199/200 [12:07<00:03,  3.91s/it]

=== Label: 0, proba true:0.7024856209754944, proba false:0.2974511384963989, change: 1
alpha: 0, tick: 1 diff: -0.9994159889920411, tot: 0.9994353102938476, proba_true: 9.66065090324264e-06, proba_false: 0.9994256496429443
alpha: 1, tick: 1 diff: -0.8686525300145149, tot: 0.9998413845896721, proba_true: 0.06559442728757858, proba_false: 0.9342469573020935
alpha: 2, tick: 1 diff: 0.9985771963838488, tot: 0.9999528338667005, proba_true: 0.9992650151252747, proba_false: 0.0006878187414258718
alpha: 1.5, tick: 0.5 diff: 0.8065481260418892, tot: 0.9999500438570976, proba_true: 0.9032490849494934, proba_false: 0.09670095890760422
alpha: 1.25, tick: 0.25 diff: -0.01562228798866272, tot: 0.9999065101146698, proba_true: 0.49214211106300354, proba_false: 0.5077643990516663
alpha: 1.5, tick: 0.25 diff: 0.8065481260418892, tot: 0.9999500438570976, proba_true: 0.9032490849494934, proba_false: 0.09670095890760422
alpha: 1.375, tick: 0.125 diff: 0.5212658792734146, tot: 0.9999320358037949, proba_true

100%|██████████| 200/200 [12:10<00:00,  3.65s/it]

=== Label: 0, proba true:0.9867413640022278, proba false:0.013222342357039452, change: 1


In [22]:
save_results(
    test_95_5_results_confounding,
    class_names=CLASS_NAMES,
    aspect_names=CONFOUNDING_ASPECT_NAMES,
    save_path=f"{CACHE_DIR}/probing_with_interventions/test_95_5/confounding_aspect_{CONFOUNDING_ASPECT_MODEL}_{CONFOUNDING_ASPECT_LAYER}.json",
)

In [23]:
test_95_5_results_confounding

{0: {0: 0.9, 1: -0.9}, 1: {0: -0.56, 1: 0.56}}